In [1]:
import os
import json
import gzip
import pandas
from tqdm import tqdm

import spacy
from spacy.matcher import Matcher


##Данные

In [2]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz

--2021-12-03 15:13:35--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36119369 (34M) [application/x-gzip]
Saving to: ‘reviews_Pet_Supplies_5.json.gz’

reviews_Pet_Supplie 100%[===================>]  34.45M  18.9MB/s    in 1.8s    

2021-12-03 15:13:37 (18.9 MB/s) - ‘reviews_Pet_Supplies_5.json.gz’ saved [36119369/36119369]



In [3]:
data = []
with gzip.open('reviews_Pet_Supplies_5.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip())['reviewText'])

In [4]:
data[1679]

"I've found that almost all cats strongly prefer to drink running water as opposed to still water. Cat eyes are attracted to motion. My cats drink much more often from a fountain than a water dish which makes me feel better about their health. Cats love this thing, the only cons are from the human maintenance side of things.Cons: A water fountain is more work than filling up a dish. You must plug it in, turn it on, clean the pump, replace the filter, and refill the resevoir often. This does get annoying but is not a problem unqiue to this fountain, and to me it's definitely worth it for my cats.The fountain is pleasant looking, a perfect size, and very quiet."

# Способы


##1. Использовать предобученную модель


Например, диппавловских бертов. В списке возможных тэгов есть PRODUCT, то есть упоминание товара, что похоже на то, что нам нужно в этой задаче. Для каких-то из их бертов нужны данные в формате CoNLL, но можно найти и модели, не требующие перевода в другой формат. 

Сложно сказать по плюсам и минусам, так как я не понимаю, как работать с этими диппавловскими конфигами (я правда очень долго пыталсь разобраться), поэтому запустить модель не могу. Но можно предположить, что так как они размечают всю последовательность в BIO-формате, то это займет сравнительно больше времени, чем другие методы. Однако качество почти наверняка будет выше.

## 2. Очень тупо


Но почему-то есть ощущение, что подействует относительно неплохо, особенно если мы хотим достать не просто упоминания, а именно брэнды:
- выделим все не первые в предложении слова с заглавной буквы, можно так же сделать POS фильтрацию;
- выберем те упоминания, которые встречаются не единожды.

Кажется, в таком случае вполне велика вероятность, что мы достанем именно бренды. Другие сущности с большой буквы - имена или города - едва ли встретятся в отзывах. Однако, например, в отзывах на товары моей категории (товары для питомцев) могут встречаться имена животных, поэтому нужен фильтр на по частотности.

Плюсы - очень легко и быстро реализовать, почти наверняка все, что останется, будут названиями.

Минусы - можем много чего не достать, например, если в названии несколько слов, а с большой буквы записано только первое; также мы будем доставать только конкретные название, а не в принципе все упоминания товара (не особо поняла, что в этой домашке будетт предпочтительнее).

## 3. По шаблонам

Я посмотрела очень много примеров отзывов и выделила/придумала такие шаблоны: 
- buy/purchase/get/use/try the/this ____
- this/these is ADJ (great/excellent/first/second/third) ______

В данном случае очень сложно придумать какой-то определенный шаблон для конкретного товара, так как товаров очень много и очень сложно составить хоть какой-то их список (товары встречаются от зубной пасты для домашних питомцев до питьевых фонтанчиков и развлекающих видео с птичками). Поэтому шаблоны достаточно размытые, и я ожидаю, что мусора будет много, да и многое мы не выделим. Но хоть что-то.

Достанем сущности, которые встречаются с такими шаблонами. Так как yargy для русского языка, то воспользуемся spaCy Matcher для английского.

С прилагательными во втором шаблоне есть некоторая проблема - непонятно, относится ли оно к названию или же это просто атрибут. Я сохраняла удаляла такие в выделенных сущностях.


In [5]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

pattern_1 = [{'LEMMA': {"IN": ["purchase", "buy", 'try', 'use']}},
             {'LEMMA': {'IN': ['the', 'this']}},
             {"POS": "ADJ", "OP": "*"},
             {'POS': {"IN": ["PROPN", "NOUN"]}, 'OP': '+'}]
matcher.add("verb", None, pattern_1)

pattern_2 = [{'LOWER': {'IN': ['this', 'these']}},
              {'LEMMA': 'be'}, 
              {'LOWER': 'a', "OP": "?"},
              {'POS': {"IN": ["NUM", 'PRON', 'DET']}, 'OP': '*'},
              {'LEMMA': {"IN": ["great", 'excellent', 'first', 'second', 'third', 'perfect']}, 'OP': '+'},
              {'POS': {"IN": ["PROPN", "NOUN"]}, 'OP': '+'}]
matcher.add("this_is", None, pattern_2)


In [6]:
matcher_matcher = Matcher(nlp.vocab)

clear_pattern = [{'POS': {"IN": ["PROPN", "NOUN"]}, 'OP': '+'}]
      
matcher_matcher.add('clear_pattern', None, clear_pattern)
    

In [7]:
match_ids = {}
clear_matches = []

for review in tqdm(data[:10000]):
    doc = nlp(review)
    matches = matcher(doc)
    spans = [doc[start:end] for _, start, end in matches]
    for span in spacy.util.filter_spans(spans):
        match = nlp(str(span))
        match_match = matcher_matcher(match)
        spans_spans = [match[start:end] for _, start, end in match_match]
        for span_span in spacy.util.filter_spans(spans_spans):
            clear_matches.append(span_span)




100%|██████████| 10000/10000 [03:50<00:00, 43.44it/s]


In [8]:
clear_matches = [str(match) for match in clear_matches]

Отфильтруем: самые частотные, потому что это мусор -уберем все, что встречается чаще 6 раз

In [9]:
import collections

In [10]:
counter=collections.Counter(clear_matches)
counts={}
for k, v in counter.items():
    counts.setdefault(v, []).append(k)

tol=6   
a = {k:v for k,v in counter.items() if v<tol}

In [11]:
a

{'1L product': 1,
 '1L size': 1,
 '640z': 1,
 ':)': 1,
 'A': 1,
 'AC50': 2,
 'AC70 filter': 1,
 'AKC brand': 1,
 'API': 1,
 'API Filstar XP': 1,
 'API Master kit': 1,
 'API strips': 1,
 'API test': 1,
 'API test kits': 1,
 'Advantage': 2,
 'Airtight Food Container': 1,
 'Ammonia': 1,
 'Andis': 1,
 'AquaClear filters': 1,
 'AquaSafe': 1,
 'Aquaclear': 3,
 'Aquaclear unit': 1,
 'BISSELL': 1,
 'BOUGHT': 1,
 'Bark Free': 1,
 'Beef': 1,
 'Big Chews': 2,
 'Bio': 1,
 'Bissell Pet Stain': 1,
 'Bissell Pre': 1,
 'Bissell Spot': 1,
 'Black Medium': 1,
 'Blueberry': 1,
 'C3': 1,
 'CET paste': 1,
 'Canidae ALS': 1,
 'Carbon': 1,
 'Carefresh Natural': 1,
 'Cat Charmer': 1,
 'Cat Tent': 1,
 'Chicken Zuke': 1,
 'Chuck': 2,
 'Chuckit': 4,
 'Chuckit Mega ball': 1,
 'Chuckit Ultra Balls': 1,
 'Chuckit medium Ultra Balls': 1,
 'Chuckit thrower': 1,
 'DIY filter media': 1,
 'DRY': 1,
 'Dawn': 2,
 'Dino': 1,
 'EI': 1,
 'Excel': 1,
 'Extreme Carpet Shampoo': 1,
 'Fetch balls': 1,
 'Flow': 1,
 'Fluval C4': 1

И даже с фильтрацией получилось очень грязно, часто выбранные шаблоны используются в целом для описания товара и некоторых его категорий/характеристик, да и в целом частотны. Без точного названия продукта сложно придумать хороший фильтр. 

Тут я увидела сообщение в чате телеграма, что можно выбрать конкретный продукт и по нему доставать бренды. Я решила выбрать filter, fountain. Первым словом в шаблоне будет the/this, потому что скорее всего название бренда будет начинаться с них. Так же я не буду рассматривать прилагательные в середине названия, потому что это редко, иначе попадает много мусоры

In [12]:
matcher_products = Matcher(nlp.vocab)
pattern = [{'LEMMA': {'IN': ['the', 'this']}},
           {'POS': {"IN": ["PROPN", "NOUN"]}, 'OP': '+'}, 
           {'LEMMA': {'IN': ['filter', 'fountain']}}]
matcher_products.add("products", None, pattern)


In [13]:
product_matches = []
for review in tqdm(data[:50000]):
    doc = nlp(review)
    matches = matcher_products(doc)
    spans = [doc[start:end] for _, start, end in matches]
    for span in spacy.util.filter_spans(spans):
        product_matches.append(span)

100%|██████████| 50000/50000 [17:54<00:00, 46.53it/s]


Уберем первое и последнее слова, потому что это this/the и сам продукт, таким образом останутся только названия

In [14]:
product_matches[:10]

[the charcoal filter,
 the carbon filters,
 The charcoal filter,
 the scent filter,
 the water filter,
 the Drinkwell Platinum fountain,
 the Petmate fountain,
 the PetMate fountain,
 the power filter,
 the power filter]

In [15]:
fountain_brands = []
filter_brands = []
for product in product_matches:
    product = str(product)
    if product.split()[-1] in ['fountain', 'fountains']:
        name = ' '.join(product.split()[1:-1])
        fountain_brands.append(name)
    else:
        name = ' '.join(product.split()[1:-1])
        filter_brands.append(name)

In [16]:
counter_fountain=collections.Counter(fountain_brands)
counts={}
for k, v in counter_fountain.items():
    counts.setdefault(v, []).append(k)

counts

{1: ['Petmate medium size water',
  'freshflow',
  'Jumbo drinking',
  'Pet Mate water',
  'Petmate Fresh Flow water',
  'Petco brand pet',
  'replacement',
  'cat water',
  'bubble',
  'Water',
  'Haagen dome',
  'Dr. Fosters water',
  'Drihnkwell',
  'Drinkwell cat',
  'petmate',
  'Platinum',
  'cat drinkwell',
  'cage water',
  'catit',
  'Hagen Catit drinking',
  'Drinkwell Pet',
  'Catit',
  'Big Dog pet'],
 2: ['Drinkwell Platinum', 'Petmate', 'PetMate', 'cat'],
 4: ['pet', 'drinking'],
 5: ['platinum'],
 6: ['drinkwell'],
 11: ['Drinkwell'],
 17: ['water']}

In [17]:
counter_filters=collections.Counter(filter_brands)
counts={}
for k, v in counter_filters.items():
    counts.setdefault(v, []).append(k)

counts

{1: ['scent',
  'biomax',
  'Aquaclear power',
  'Aqueon',
  'Marineland HOB',
  'Foam',
  'Fluval canister',
  'box',
  'Marineland Biowheel',
  'Marineland',
  'intake',
  'round pre',
  'tank style',
  'marineland',
  'attachment',
  'premade',
  'factory Seaclear',
  'Diatom',
  'Marineland Canister',
  'Marineland Emporer',
  'C',
  'Fluval FX-5 canister',
  'fish',
  'Rena XP series',
  'ZooMed canister',
  'Marina carbon',
  'Marina gravel',
  'Aqua Clear',
  'tetra',
  'Fluval Spec',
  'cansiter',
  'sunsun cansiter',
  'edge',
  'Classice Eheim',
  'HEPA air',
  'tom',
  'whisper',
  'Tetra power',
  'mesh',
  'Bio Bag',
  'size',
  'description',
  'Fluval Underwater',
  'plastic liter',
  'litter',
  'hanging',
  'eheim classic series',
  'Rena FilStar XP3 canister',
  'True Air',
  'Drinkwell',
  'fountain',
  'Platinum',
  'filtration',
  'magnum power',
  'NOISY',
  'rite size E',
  'Emperor',
  'line',
  'undergravel',
  'bubble',
  'Penquin',
  'penguin power',
  'Fresh

# N-граммы

Для поиска n-грамм выберем 5 брэндов поилок и фильтров вручную, потому что мусора все еще не мало.

Fountain:
- Drinkwell
- Platinum
- PetMate
- Catit
- Jumbo

Filter:
- AquaClear
- Whisper
- Marineland
- Fluval
- Penguin

In [77]:
import nltk
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()


corpus = ' '.join([d.lower() for d in data])
tokens = nltk.wordpunct_tokenize(corpus)
finder_bigrams_filter = BigramCollocationFinder.from_words(tokens)
finder_trigrams_filter = TrigramCollocationFinder.from_words(tokens)

finder_bigrams_fontain = BigramCollocationFinder.from_words(tokens)
finder_trigrams_fontain  = TrigramCollocationFinder.from_words(tokens)

In [81]:
filter_filter = lambda *w: 'platinum' not in w and 'drinkwell' not in w and 'catit' not in w and 'jumbo' not in w and 'petmate' not in w
finder_bigrams_filter.apply_freq_filter(3) # фильтр на нграммы, которые встречаются чаще 3
finder_trigrams_filter.apply_freq_filter(3)

finder_bigrams_filter.apply_ngram_filter(filter_filter) # нграммы с выбранными брендами
finder_trigrams_filter.apply_ngram_filter(filter_filter)

print('1. Log likelihood\nБиграммы:', finder_bigrams_filter.nbest(bigram_measures.likelihood_ratio, 20))
print('Триграммы:', finder_trigrams_filter.nbest(trigram_measures.likelihood_ratio, 20))

print('2. PMI\nБиграммы:', finder_bigrams_filter.nbest(bigram_measures.pmi, 20))
print('Триграммы:', finder_trigrams_filter.nbest(trigram_measures.pmi, 20))

print('3. Jaccard\nБиграммы:', finder_bigrams_filter.nbest(bigram_measures.jaccard, 20))
print('Триграммы:', finder_trigrams_filter.nbest(trigram_measures.jaccard, 20))


1. Log likelihood
Биграммы: [('drinkwell', 'platinum'), ('drinkwell', 'fountain'), ('the', 'drinkwell'), ('drinkwell', '360'), ('platinum', 'fountain'), ('drinkwell', 'hy'), ('the', 'petmate'), ('drinkwell', 'fountains'), ('jumbo', 'size'), ('the', 'catit'), ('hagen', 'catit'), ('platinum', 'pet'), ('the', 'jumbo'), ('catit', 'fountain'), ('petmate', 'deluxe'), ('drinkwell', 'original'), ('catit', 'design'), ('the', 'platinum'), ('catit', 'senses'), ('petmate', 'fountain')]
Триграммы: [('drinkwell', "'", 's'), ('petmate', "'", 's'), ('platinum', "'", 's'), ('catit', "'", 's'), ('platinum', '.', 'i'), ('drinkwell', '.', 'i'), ('petmate', '.', 'i'), ('jumbo', '&#', '34'), ('platinum', ',', 'but'), ('drinkwell', ',', 'but'), ('34', ';', 'jumbo'), ('in', 'the', 'drinkwell'), ('in', 'the', 'petmate'), ('in', 'the', 'catit'), ('in', 'the', 'jumbo'), ('of', 'the', 'drinkwell'), ('of', 'the', 'petmate'), ('of', 'the', 'catit'), ('of', 'the', 'jumbo'), ('of', 'the', 'platinum')]
2. PMI
Биграммы

In [82]:
filter_fontain = lambda *w: 'aquaclear' not in w and 'whisper' not in w and 'marineland' not in w and 'fluval' not in w and 'penguin' not in w 
finder_bigrams_fontain.apply_freq_filter(3) # фильтр на нграммы, которые встречаются чаще 3
finder_trigrams_fontain.apply_freq_filter(3)

finder_bigrams_fontain.apply_ngram_filter(filter_fontain) # нграммы с выбранными брендами
finder_trigrams_fontain.apply_ngram_filter(filter_fontain)

print('1. Log likelihood\nБиграммы:', finder_bigrams_fontain.nbest(bigram_measures.likelihood_ratio, 20))
print('Триграммы:', finder_trigrams_fontain.nbest(trigram_measures.likelihood_ratio, 20))

print('2. PMI\nБиграммы:', finder_bigrams_fontain.nbest(bigram_measures.pmi, 20))
print('Триграммы:', finder_trigrams_fontain.nbest(trigram_measures.pmi, 20))

print('3. Jaccard\nБиграммы:', finder_bigrams_fontain.nbest(bigram_measures.jaccard, 20))
print('Триграммы:', finder_trigrams_fontain.nbest(trigram_measures.jaccard, 20))


1. Log likelihood
Биграммы: [('tetra', 'whisper'), ('fluval', 'edge'), ('the', 'fluval'), ('aquaclear', '50'), ('fluval', 'spec'), ('fluval', '306'), ('aquaclear', '20'), ('fluval', 'fx5'), ('fluval', '305'), ('the', 'aquaclear'), ('aquaclear', '110'), ('an', 'aquaclear'), ('aquaclear', '70'), ('my', 'fluval'), ('whisper', 'quiet'), ('whisper', 'air'), ('fluval', '406'), ('fluval', 'chi'), ('fluval', 'canister'), ('penguin', '350')]
Триграммы: [('fluval', "'", 's'), ('marineland', "'", 's'), ('aquaclear', "'", 's'), ('fluval', '.', 'i'), ('marineland', '.', 'i'), ('aquaclear', '.', 'i'), ('whisper', '.', 'i'), ('whisper', '&#', '34'), ('marineland', ',', 'but'), ('fluval', ',', 'but'), (',', 'but', 'fluval'), ('34', ';', 'whisper'), ('34', ';', 'marineland'), ('in', 'the', 'fluval'), ('in', 'the', 'aquaclear'), ('in', 'the', 'penguin'), ('of', 'the', 'fluval'), ('of', 'the', 'aquaclear'), ('of', 'the', 'marineland'), ('of', 'the', 'whisper')]
2. PMI
Биграммы: [('marineland', 'stealthpr

Видно, что мера log likelihood оценивает высоко не то, что нам нужно, особенно для триграмм: в топе оказываются очень частотные, но бесполезные нграмы. Для PMI и Jaccard результаты выглядят так, будто это похоже на какие-то названия отдельных товаров бренда. Я возьму топ по Jaccard для товаров filters и топ по PMI для fontains.

Я не особо поняла, что нужно группировать и как это связано с метриками с предыдущего шага. Я возьму топ40 по выбранной метрике и их сгруппирую по бренду.

In [85]:
filter_bigrams = finder_bigrams_filter.nbest(bigram_measures.jaccard, 40)
filter_trigrams = finder_trigrams_filter.nbest(trigram_measures.jaccard, 40)

fontain_bigrams = finder_bigrams_fontain.nbest(bigram_measures.pmi, 40)
fontain_trigrams = finder_trigrams_fontain.nbest(trigram_measures.pmi, 40)


In [96]:
dict_filter = {'drinkwell': [],
     'platinum': [],
     'jumbo': [],
     'catit': [],
     'petmate': []}
for ngram in filter_bigrams+filter_trigrams:
    for key in dict_filter.keys(): 
        if key in ngram:
            dict_filter[key].append(' '.join(ngram))

dict_fontain = {'aquaclear': [],
     'whisper': [],
     'marineland': [],
     'fluval': [],
     'penguin': []}
for ngram in fontain_bigrams+fontain_trigrams:
    for key in dict_fontain.keys(): 
        if key in ngram:
            dict_fontain[key].append(' '.join(ngram))

In [101]:
def print_nice(d):
    for key in d.keys():
        print(key, '\n---')
        for name in d[key]:
            print(name)
        print('................\n\n')


In [103]:
print_nice(dict_filter)

drinkwell 
---
drinkwell platinum
drinkwell 360
drinkwell fountain
drinkwell hy
drinkwell fountains
platinum drinkwell
drinkwell original
original drinkwell
drinkwell rectangle
drinkwell platinum fountain
drinkwell 360 stainless
stainless steel drinkwell
drinkwell rectangle multi
steel drinkwell 360
.--- drinkwell rectangle
drinkwell 360 fountain
drinkwell platinum pet
platinum drinkwell fountain
drinkwell 360 +
drinkwell platinum premium
drinkwell stainless steel
drinkwell platinum drinking
drinkwell pet fountain
original drinkwell fountain
drinkwell original fountain
second drinkwell fountain
fountain .--- drinkwell
drinkwell original pet
drinkwell fountain premium
drinkwell hy -
drinkwell cleaning kit
................


platinum 
---
drinkwell platinum
thedrinkwell platinum
platinum fountain
platinum drinkwell
adrinkwell platinum
platinum fountains
drinkwell platinum fountain
platinum pet fountain
drinkwell platinum pet
platinum drinkwell fountain
drinkwell platinum premium
drinkwel

In [102]:
print_nice(dict_fontain)

aquaclear 
---
aquaclear 110
aquaclear 70
aquaclear biomax
aquaclear hob filters
aquaclear 70 power
aquaclear 50 foam
an aquaclear 110
................


whisper 
---
tetra whisper
tetra whisper bio
tetra whisper 60
tetra whisper air
tetra whisper pump
tetra whisper filter
whisper air pumps
whisper air pump
................


marineland 
---
marineland stealthpro
marineland crescent
marineland stealth
marineland biowheel
marineland eclipse
marineland magnum
marineland penguin
marineland emperor
marineland 350
marineland 400
marineland magnum 350
marineland penguin 100
marineland activated carbon
marineland biowheel filter
marineland double bright
marineland single bright
................


fluval 
---
fluval q2
fluval 306
fluval 304
fluval 88g
fluval 404
fluval 405
fluval spec
fluval 305
fluval 206
fluval c3
fluval fx5
fluval 205
fluval c4
fluval u2
fluval 106
fluval 406
fluval c2
fluval cf
fluval biomax
fluval prefilter
fluval spec v
fluval spec iii
fluval c4 hob
fluval co2 88
fluval 